In [1]:
%pip install ultralytics opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import shutil
import random
import cv2
from ultralytics import YOLO


In [3]:
DATASET_PATH = "clothing-detection-dataset"
IMAGE_DIR = os.path.join(DATASET_PATH, "images")
LABEL_DIR = os.path.join(DATASET_PATH, "labels")
SPLIT_DIR = os.path.join(DATASET_PATH, "split_dataset")
if not os.path.exists(IMAGE_DIR) or not os.path.exists(LABEL_DIR):
    raise FileNotFoundError("Folder not found")

print("Found Dataset:", DATASET_PATH)

Found Dataset: clothing-detection-dataset


In [4]:
folders = ["train/images", "train/labels", "val/images", "val/labels"]
for folder in folders:
    os.makedirs(os.path.join(SPLIT_DIR, folder), exist_ok=True)

In [5]:
image_files = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith((".jpg"))]

if len(image_files) == 0:
    raise ValueError("No image in folders")

random.shuffle(image_files)

In [6]:
split_ratio = 0.8
train_files = image_files[: int(len(image_files) * split_ratio)]
val_files = image_files[int(len(image_files) * split_ratio) :]

def copy_data(file_list, mode):
    for img in file_list:
        label = os.path.splitext(img)[0] + ".txt"
        shutil.copy(os.path.join(IMAGE_DIR, img), os.path.join(SPLIT_DIR, f"{mode}/images", img))
        label_src = os.path.join(LABEL_DIR, label)
        if os.path.exists(label_src):
            shutil.copy(label_src, os.path.join(SPLIT_DIR, f"{mode}/labels", label))

copy_data(train_files, "train")
copy_data(val_files, "val")

print(f"Split Done, Train: {len(train_files)}, Validation: {len(val_files)}")


Split Done, Train: 2046, Validation: 512


In [7]:
yaml_path_original = os.path.join(DATASET_PATH, "data.yaml")
yaml_path_new = os.path.join(DATASET_PATH, "yolo_data.yaml")

with open(yaml_path_original, "r") as f:
    yaml_content = f.read()

yaml_content = yaml_content.replace("images", f"{SPLIT_DIR}/train/images")

In [ ]:
with open(yaml_path_new, "w") as f:
    f.write(f"""
train: clothing-detection-dataset/split_dataset/train/images
val:clothing-detection-dataset/split_dataset/val/images
nc: 7
names:
  0: crop_top
  1: sleeveless_top
  2: transparent_top
  3: ripped_jeans
  4: short_medium_skirt
  5: shorts
  6: opened_foot
""")
    

In [ ]:
model = YOLO("yolov8s.pt")
model.train(
    data=yaml_path_new,
    epochs=50,
    imgsz=640,
    batch=16,
    lr0=0.01, lrf=0.1,
    momentum=0.937,
    weight_decay=0.0005,
    optimizer="SGD", 
)

Ultralytics 8.3.235  Python-3.13.9 torch-2.9.1+cpu CPU (12th Gen Intel Core i7-12650H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=clothing-detection-dataset\yolo_data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=100, perspective=0.0, plo

AttributeError: 'DetectionModel' object has no attribute 'evolve'

In [ ]:
model_path = "runs/detect/train/weights/best.pt"
model = YOLO(model_path)

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    annotated_frame = results[0].plot()

    if len(results[0].boxes) > 0:
        status_text = "Tidak Layak Dipakai"
        color = (0, 0, 255)
    else:
        status_text = "Layak Dipakai"
        color = (0, 255, 0)

    cv2.putText(annotated_frame, status_text, (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    cv2.imshow("Real-Time Clothing Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()